In [ ]:
! lamin load scprint

In [2]:
%load_ext autoreload
%autoreload 2

import lamindb as ln
import lnschema_bionty as lb

import pandas as pd

from lightning.pytorch import Trainer, seed_everything
seed_everything(42, workers=True)

from scprint import scPrint
from scprint.utils import getBiomartTable

from scdataloader import Dataset
from scdataloader import DataModule
from scprint.dataloader import embed
from scdataloader.utils import load_genes
from scprint.dataloader import Collator

import torch 
torch.set_float32_matmul_precision('medium')

lb.settings.organism = "human"

INFO: Seed set to 42
2024-01-28 15:26:28,774:INFO - Seed set to 42


No module named 'triton'
can't use flash attention and triton kernel,        you likely don't have the right hardware or didn't         make the right installation


/opt/homebrew/Caskroom/miniconda/base/envs/nscprint/lib/python3.10/site-packages/gget/gget_setup.py:252: DeprecationWarning: invalid escape sequence '\/'
  command = """
/opt/homebrew/Caskroom/miniconda/base/envs/nscprint/lib/python3.10/site-packages/gget/gget_setup.py:263: DeprecationWarning: invalid escape sequence '\/'
  "/", "\/"
/opt/homebrew/Caskroom/miniconda/base/envs/nscprint/lib/python3.10/site-packages/gget/gget_setup.py:271: DeprecationWarning: invalid escape sequence '\/'
  command = """
/opt/homebrew/Caskroom/miniconda/base/envs/nscprint/lib/python3.10/site-packages/gget/gget_setup.py:282: DeprecationWarning: invalid escape sequence '\/'
  "/", "\/"


ImportError: cannot import name 'load_genes' from 'scdataloader.utils' (/opt/homebrew/Caskroom/miniconda/base/envs/nscprint/lib/python3.10/site-packages/scdataloader/utils.py)

In [1]:
## Gene embeddings
# embeddings = embed(genedf=genedf,
#     organism="homo_sapiens",
#     cache=True,
#     fasta_path="/tmp/data/fasta/",
#     embedding_size=1024,)
# embeddings.to_parquet('../data/temp/embeddings.parquet')
embeddings = pd.read_parquet('../data/temp/embeddings.parquet')
embeddings.columns = ['emb_'+str(i) for i in embeddings.columns]
# and annotations
biomart = getBiomartTable(attributes=['start_position', 'chromosome_name']).set_index('ensembl_gene_id')
biomart = biomart.loc[~biomart.index.duplicated(keep='first')]
biomart = biomart.sort_values(by=['chromosome_name', 'start_position'])
# and location
c = []
i = 0
prev_position = -100000
prev_chromosome = None
for _, r in biomart.iterrows():
    if r['chromosome_name'] != prev_chromosome or r['start_position'] - prev_position > 10_000:
        i += 1
    c.append(i)
    prev_position = r['start_position']
    prev_chromosome = r['chromosome_name']
print(f'reduced the size to {len(set(c))/len(biomart)}')
biomart['pos'] = c

NameError: name 'pd' is not defined

In [ ]:
# OR directly load the dataset
name="preprocessed dataset"
dataset = ln.Collection.filter(name=name).first()
dataset.artifacts.count()

91

In [ ]:
# TODO: drop tissue & dev stage until part or is taken in account

hierarchical_labels = [
    "cell_type_ontology_term_id",
    #"tissue_ontology_term_id",
    "disease_ontology_term_id",
    #"development_stage_ontology_term_id",
    "assay_ontology_term_id",
    'self_reported_ethnicity_ontology_term_id',
]

labels_weighted_sampling = hierarchical_labels+[
    'sex_ontology_term_id',
    "organism_ontology_term_id",
]

all_labels = labels_weighted_sampling+[
    #'dataset_id',
    #'cell_culture',
    "heat_diff",
    "total_counts",
    "nnz",
    "dpt_group",
]

In [ ]:
mdataset = Dataset(dataset, organisms=["NCBITaxon:9606"], obs=all_labels, clss_to_pred=labels_weighted_sampling, hierarchical_clss=hierarchical_labels)
print(mdataset)

won't do any check but we recommend to have your dataset coming from local storage



In [ ]:
# we might want not to order the genes by expression (or do it?)
# we might want to not introduce zeros and 

In [ ]:
col = Collator(organisms=["NCBITaxon:9606",], labels=all_labels, genelist=embeddings.index.tolist(), max_len=1000, add_zero_genes=100, org_to_id={'NCBITaxon:9606': mdataset.encoder['organism_ontology_term_id']['NCBITaxon:9606']})#mdataset.encoder['organism_ontology_term_id'])

In [ ]:
datamodule = DataModule(mdataset, label_to_weight=labels_weighted_sampling, collate_fn=col, batch_size=32, num_workers=4)
datamodule.setup()

In [ ]:
for i in datamodule.train_dataloader():
    break

In [ ]:
d_model = 128
m = torch.nn.AdaptiveAvgPool1d(d_model)
sembeddings = pd.DataFrame(
    data=m(torch.tensor(embeddings.values)), index=embeddings.index, columns=[f'emb_{i}' for i in range(d_model)]
)

In [ ]:
labels = {k: len(v) for k, v in mdataset.class_topred.items()}

cls_hierarchies = {}
for k, dic in mdataset.class_groupings.items():
    rdic = {}
    for sk, v in dic.items():
        rdic[mdataset.encoder[k][sk]] = [mdataset.encoder[k][i] for i in list(v)]
    cls_hierarchies[k] = rdic

df = sembeddings.join(biomart,how="inner")

genedf = load_genes(['NCBITaxon:9606'])
df = df.loc[genedf[genedf.index.isin(df.index)].index]

In [ ]:
model = scPrint(
    genes = df.index.tolist(),
    d_model = d_model,
    nhead = 4,
    d_hid = d_model,
    nlayers = 2,
    layers_cls = [],
    labels = labels,
    cls_hierarchy = cls_hierarchies,
    dropout= 0.2,
    transformer = "fast",
    use_precpt_gene_emb = df.iloc[:,:d_model].values.astype(float),
    gene_pos_enc = df['pos'].tolist(),
    mvc_decoder = "inner product",
)

2024-01-27 23:28:52,033:INFO - Created a temporary directory at /tmp/tmp82od2gry
2024-01-27 23:28:52,035:INFO - Writing /tmp/tmp82od2gry/_remote_module_non_scriptable.py
2024-01-27 23:28:52,035:INFO - Writing /tmp/tmp82od2gry/_remote_module_non_scriptable.py


In [ ]:
# create a function to transform an scGPT checkpoint to an scPrint's
# ckpt = torch.load("../../scGPT/save/model_e6.pt")
# scPrint.load_from_checkpoint("../../scGPT/save/model_e6.pt")

In [ ]:
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.loggers import WandbLogger

#wandb_logger = WandbLogger(project="scprint_test", save_dir="../data/tensorboard")
#wandb_logger.watch(model)

tlogger = TensorBoardLogger(save_dir="../data/tensorboard")
tlogger.log_graph(model, i)


In [ ]:
from lightning.pytorch.profilers import PyTorchProfiler
pytorch_prof = PyTorchProfiler("../data/tensorboard", emit_nvtx=False, group_by_input_shape=True, record_shapes=True, profile_memory=True, with_stack=True, on_trace_ready=torch.profiler.tensorboard_trace_handler("../data/tensorboard/"),)

In [ ]:
# sets seeds for numpy, torch and python.random.
trainer = Trainer(precision=16, profiler="simple", fast_dev_run=1000, logger=tlogger)#, wandb_logger], limit_train_batches=0.1, limit_test_batches=0.03, limit_val_batches=0.03)


INFO: Using 16bit Automatic Mixed Precision (AMP)
2024-01-27 23:28:56,458:INFO - Using 16bit Automatic Mixed Precision (AMP)
2024-01-27 23:28:56,458:INFO - Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
2024-01-27 23:28:56,794:INFO - GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
2024-01-27 23:28:56,797:INFO - TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
2024-01-27 23:28:56,800:INFO - IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
2024-01-27 23:28:56,803:INFO - HPU available: False, using: 0 HPUs
INFO: Running in `fast_dev_run` mode: will run the requested loop using 1000 batch(es). Logging and checkpointing is suppressed.
2024-01-27 23:28:56,807:INFO - Running in `fast_dev_run` mode: will run the requested loop using 1000 batch(es). Logging and checkpointing is suppressed.


In [85]:
model.labels = {}
model.expr_decoder.nfirst_labels_to_skip=3

In [20]:
trainer.fit(model, datamodule=datamodule)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2024-01-27 23:42:54,990:INFO - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
   | Name          | Type                   | Params
----------------------------------------------------------
0  | gene_encoder  | GeneEncoder            | 4.3 M 
1  | expr_encoder  | ContinuousValueEncoder | 512   
2  | pos_encoder   | PositionalEncoding     | 0     
3  | label_encoder | BatchLabelEncoder      | 1.4 K 
4  | time_encoder  | ContinuousValueEncoder | 512   
5  | depth_encoder | ContinuousValueEncoder | 512   
6  | transformer   | TransformerEncoder     | 796 K 
7  | expr_decoder  | ExprDecoder            | 50.1 K
8  | cls_decoders  | ModuleDict             | 29.9 K
9  | mvc_decoder   | MVCDecoder             | 65.7 K
10 | sim           | Similarity             | 0     
----------------------------------------------------------
945 K     Trainable params
4.3 M     Non-trainable params
5.3 M     Total params
21.134    Total estimated model param

Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir="../data/tensorboard"

In [ ]:
# TODO: log embeddings of N cells + cell type & disease & tissue & sex & predicted version as scanpy plot and log them to wandb 

In [ ]:
2.66 - 128 - noclass - 64 - 8
4.22 - 128 - noclass - 32 - 4
4.50 - 128 - noclass - 32 - 8
9.00 - 128 - 1 MLM   - 32 - 8
5.80 - 256 - 1 MLM   - 32 - 8
2.80 - 256 - noclass - 32 - 8
2.82 - 256 - noclass - 32 - 4
1.70 - 256 - all   - 32 - 8
# I could go 2 times faster with better engineering
# I need to rework the classification task
 - 128 - all - 32 - 8

In [62]:
wandb_logger.finalize(status="aborted")
torch.cuda.empty_cache()

In [ ]:
from lightning.pytorch.tuner import Tuner
tuner = Tuner(trainer)
tuner.scale_batch_size(model, mode="power") #default
tuner.lr_find(model)

In [ ]:
# TODO: debug how much time is spent on class prediction (0.5 day)
# TODO: test unseen genes (do we see much being kept after filtering and stuff) (0.5 day)
# TODO: visualize and assess the embeddings (0.5 day)
----
# TODO: add model checkpointing (0.5 day) 
# TODO: add method to run the model as a script (0.5 day) ---
# TODO: create env and copy data to maestro (0.5 day)
# TODO: create script and test it on maestro (0.5 day)
# TODO: connect with maestro people to ask for longer compute time 
# TODO: do the same to jean zay (0.5 day)
------
# TODO: make a model benchmark package (continue from where I left off) (4 days)
# TODO: make a task function & make a benchmark function (1 day) (*denoising, *classification, *embeddings, *perturbation prediction)
# TODO: define the test set as a subcollection of datasets (print them and save as a file somewhere too) (0.5 day)
------
# TODO: debug the gene embedding creation
# TODO: create embedding & make it work for the 4-5 species in the dataset (1 days) 
# TODO: debug the timepoint problem (2 days)
# TODO: find the neighboors and next time point cells (1 days)
# TODO: create a version with next time point and neighboors task (1 days)
# TODO: make a trajectory prediction task (predict future cell type/s, expression) and benchmark (similarity to known future cell, similarity to known future expression) (1 days)
------
# TODO: log more info with .log() during training (the model architecture) (some weights? some gradients?) (0.5 day)
# TODO: run a large training on maestro (0.5 day)
------
# TODO: add KO & drug datasets
# TODO: create a version with KO and drug effect prediction